In [1]:
import os 
os.chdir('../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd
import numpy as np
import csv
import ky_utils.ky_baseline_functions as base

Current working directory is now:  /home/jovyan/work/cynthia/interpretable-machine-learning/ky-fl-combined/ky-model


In [2]:
KY_data = pd.read_csv("/home/jovyan/work/cynthia/data/ky-fl-data/KY-cleaned.csv").sort_values('person_id')
FL_data = pd.read_csv("/home/jovyan/work/cynthia/data/ky-fl-data/FL-cleaned.csv").sort_values('person_id')

## split x and y 
KY_X = KY_data.loc[:,:'current_violence20']
KY_Y = KY_data['general_two_year'].values
FL_X = FL_data.loc[:,:'current_violence20']
FL_Y = FL_data['general_two_year'].values

### Model

In [3]:
#### Logistic
c = [1e-4, 1e-3, 1e-2, 1e-1, 1]
logistic_summary = base.Logistic(KY_X, KY_Y, FL_X, FL_Y,C=c,seed=816)

#### Lasso
c = [1e-4, 1e-3, 1e-2, 1e-1, 1]
lasso_summary = base.Lasso(KY_X, KY_Y, FL_X, FL_Y,C=c,seed=816)

#### LinearSVM
c = [1e-4, 1e-3, 1e-2, 1e-1, 1]
svm_summary = base.LinearSVM(KY_X, KY_Y, FL_X, FL_Y,C=c,seed=816)

#### Random Forest
n_estimators =  [100,150,200]
depth = [7,8,9]
rf_summary = base.RF(KY_X, KY_Y, FL_X, FL_Y, 
                     depth=depth, 
                     estimators=n_estimators,
                     seed=816)

#### XGBoost
learning_rate = [0.1]
depth = [4,5,6]
n_estimators = [100,150]
xgb_summary = base.XGB(KY_X, KY_Y, FL_X, FL_Y,
                       learning_rate=learning_rate, 
                       depth=depth, 
                       estimators=n_estimators,
                       seed=816)

#### save results
summary_general2_ky_model = {"Logistic": logistic_summary,
                             "Lasso": lasso_summary,
                             "LinearSVM": svm_summary,
                             "RF": rf_summary,
                             "XGBoost": xgb_summary}

In [4]:
results = []
for model_name, model_summary in summary_general2_ky_model.items():
    results.append([model_name, 
                    np.mean(model_summary['KY_validation']), 
                    np.mean(model_summary['auc_diff']), 
                    np.mean(model_summary['FL_score'])])
results

[['Logistic', 0.7392294483425521, 0.00029675931427255444, 0.6147708297026251],
 ['Lasso', 0.7392129150416151, 0.00028959536886488115, 0.6142822288059062],
 ['LinearSVM', 0.7401034824082645, 0.0002550433400892338, 0.6095861256779522],
 ['RF', 0.7519609087405618, 0.008697479415809983, 0.6193611009891369],
 ['XGBoost', 0.7563922984618646, 0.01284520267933551, 0.6169377419371874]]

In [5]:
path = "./results/baselines/two-year/"
results = [["", "Logistic", "Lasso", "Linear SVM", "Random Forest", "XGBoost"],
           ["General",             
            np.str(round(np.mean(logistic_summary['FL_score']),3)) + " (" + np.str(round(np.std(logistic_summary['FL_score']),3)) + ")", 
            np.str(round(np.mean(lasso_summary['FL_score']), 3)) + " (" + np.str(round(np.std(lasso_summary['FL_score']),3)) + ")", 
            np.str(round(np.mean(svm_summary['FL_score']),3)) + " (" + np.str(round(np.std(svm_summary['FL_score']),3)) + ")", 
            np.str(round(np.mean(rf_summary['FL_score']),3)) + " (" + np.str(round(np.std(rf_summary['FL_score']),3)) + ")", 
            np.str(round(np.mean(xgb_summary['FL_score']),3)) + " (" + np.str(round(np.std(xgb_summary['FL_score']),3)) + ")"]]
with open(path + 'two-year-ky-baseline-fl-score.csv', 'w') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)

In [6]:
path = "./results/baselines/two-year/"
results = [["", "Logistic", "Lasso", "Linear SVM", "Random Forest", "XGBoost"],
           ["General",             
            np.str(round(np.mean(logistic_summary['KY_score']),3)) + " (" + np.str(round(np.std(logistic_summary['KY_score']),3)) + ")", 
            np.str(round(np.mean(lasso_summary['KY_score']), 3)) + " (" + np.str(round(np.std(lasso_summary['KY_score']),3)) + ")", 
            np.str(round(np.mean(svm_summary['KY_score']),3)) + " (" + np.str(round(np.std(svm_summary['KY_score']),3)) + ")", 
            np.str(round(np.mean(rf_summary['KY_score']),3)) + " (" + np.str(round(np.std(rf_summary['KY_score']),3)) + ")", 
            np.str(round(np.mean(xgb_summary['KY_score']),3)) + " (" + np.str(round(np.std(xgb_summary['KY_score']),3)) + ")"]]
with open(path + 'two-year-ky-baseline-ky-score.csv', 'w') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)